# Compute urban form metrics - Urban audit

In [33]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin
import copy
from sklearn import linear_model
import math
from os import listdir
import rioxarray as rxr
from shapely.geometry import mapping
import os
import matplotlib.pyplot as plt


In [34]:
path_folder = 'C:/Users/charl/OneDrive/Bureau/EUBUCCO/'

In [35]:
res = []
# Iterate directory
for file in listdir(path_folder + "Data/EUBUCCO_by_UA/"):
    # check only text files
    if (file.endswith('.shp')):
        res.append(file)
print(len(res))

938


In [36]:
urban_form_metrics = pd.DataFrame(index=res, columns=['n_buildings', 'n_pop', 'UA', 'BA', 'BV', 'density_b',
                                  'avg_height', 'vol_per_cap', 'volume_profile', 'footprint_profile', 'centrality1', 'centrality2', 'agglo_spag', 'missing_height'])


In [37]:
URB_A = gpd.read_file(
    path_folder + "Data/ref-urau-2018-01m.shp/URAU_RG_01M_2018_3035_CITIES.shp")
URB_A = URB_A.to_crs('epsg:3035')


In [38]:
def crop_density(city_name, raster_name):
    try:
        crop_extent = URB_A.loc[URB_A.URAU_CODE == city_name[:-4], :]
        raster = rxr.open_rasterio(path_folder + raster_name)
        tiff_clipped = raster.rio.clip(
            crop_extent.geometry.apply(mapping), crop_extent.crs)
        tiff_clipped.data[tiff_clipped.data < 0] = 0
        pop_raster = (np.nansum(tiff_clipped))
    except:
        pop_raster = 0
    return pop_raster


In [60]:
for city_name in res[677:]:
    print(city_name)
    city_name = str(city_name)
    gdf = gpd.read_file(path_folder + "Data/EUBUCCO_by_UA/" + city_name)
    gdf["buildings_area"] = gdf.area
    gdf["height"] = gdf["height"].astype(float)
    gdf["volume"] = gdf["buildings_area"] * gdf["height"]
    if len(gdf)>0:
        missing_height = 100 * sum(np.isnan(gdf["height"]))/len(gdf)
    else:
        missing_height = 100
    gdf = gdf.loc[(gdf["buildings_area"] > 4) & (
        gdf["buildings_area"] < 100000) & (gdf["height"] > 2) & (gdf["height"] < 320), :]
    
    if ((missing_height < 95) & (len(gdf)>0)):
        # 1. DESCRIPTIVE STATISTICS OVER THE DATABASE

        nb_tot_buildings = len(gdf)

        # 2. PHYSICAL CHARACTERISTICS OF THE BUILT ENVIRONMENT

        BA = np.nansum(gdf["buildings_area"])
        gdf["volume"] = gdf["buildings_area"] * gdf["height"]
        BV = np.nansum(gdf["volume"])

        pop_raster1 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C21/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C21.tif')
        pop_raster2 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C20/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C20.tif')
        pop_raster3 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C19/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C19.tif')
        pop_raster4 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C18/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R5_C18.tif')
        pop_raster5 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C21/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C21.tif')
        pop_raster6 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C20/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C20.tif')
        pop_raster7 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C19/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C19.tif')
        pop_raster8 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C18/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R4_C18.tif')
        pop_raster9 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C21/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C21.tif')
        pop_raster10 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C20/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C20.tif')
        pop_raster11 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C19/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C19.tif')
        pop_raster12 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C18/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R3_C18.tif')
        pop_raster13 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C21/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C21.tif')
        pop_raster14 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C20/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C20.tif')
        pop_raster15 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C19/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C19.tif')
        pop_raster16 = crop_density(city_name,
            'Data/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C18/GHS_POP_E2015_GLOBE_R2022A_54009_1000_V1_0_R2_C18.tif')

        N = pop_raster1 + pop_raster2 + pop_raster3 + pop_raster4 + pop_raster5 + pop_raster6 + pop_raster7 + pop_raster8 + \
            pop_raster9 + pop_raster10 + pop_raster11 + pop_raster12 + \
            pop_raster13 + pop_raster14 + pop_raster15 + pop_raster16
        UA = URB_A.loc[URB_A.URAU_CODE == city_name[:-4], :].area
        density_buildings = BA/UA.squeeze()
        avg_height = BV/BA
        volume_per_capita = BV/N

        # SAVE

        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'n_buildings'] = nb_tot_buildings
        urban_form_metrics.loc[urban_form_metrics.index == city_name, 'n_pop'] = N
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'UA'] = UA.squeeze()
        urban_form_metrics.loc[urban_form_metrics.index == city_name, 'BA'] = BA
        urban_form_metrics.loc[urban_form_metrics.index == city_name, 'BV'] = BV
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'density_b'] = density_buildings
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'avg_height'] = avg_height
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'vol_per_cap'] = volume_per_capita
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'missing_height'] = missing_height
    
        # 3. CENTRALITY/MONOCENTRICITY

        gdf_centroid = copy.deepcopy(gdf)
        gdf_centroid["geometry"] = gdf_centroid.centroid

        cx = np.average(gdf_centroid["geometry"].x[~np.isnan(
            gdf_centroid["volume"])], weights=gdf_centroid["volume"][~np.isnan(gdf_centroid["volume"])])
        cy = np.average(gdf_centroid["geometry"].y[~np.isnan(
            gdf_centroid["volume"])], weights=gdf_centroid["volume"][~np.isnan(gdf_centroid["volume"])])
        center_gdf2 = gpd.GeoSeries((Point(cx, cy)))

        # centrality_index
        gdf["distance_center"] = gdf.distance(center_gdf2.geometry[0])
        avg_dist = np.average(a=gdf["distance_center"][~np.isnan(
            gdf["volume"])], weights=gdf["volume"][~np.isnan(gdf["volume"])])
        R1 = np.sqrt(np.nansum(gdf["buildings_area"])/np.pi)
        R2 = np.sqrt(UA/np.pi)

        centrality_index1 = avg_dist/R1
        centrality_index2 = avg_dist/R2

        # SAVE

        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'centrality1'] = centrality_index1
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'centrality2'] = centrality_index2.squeeze()

        # density profile

        max_dist = math.ceil(np.nanmax(gdf["distance_center"])/1000)
        buffer_df = pd.DataFrame(columns=[
                                 "dist_center", "volume", "footprint", "area_buffer"], index=np.arange(1, max_dist+1))
        buffer_df.dist_center = buffer_df.index
        for i in buffer_df.index:
            print(i)
            buffer = center_gdf2.buffer(i*1000)
            buffer_df.area_buffer.loc[buffer_df.dist_center == i] = buffer[0].intersection(
                URB_A.loc[URB_A.URAU_CODE == city_name[:-4], :].geometry.iloc[0]).area

            inter_buffer = sjoin(gdf, gpd.GeoDataFrame(
                geometry=buffer), how='inner')
            buffer_df.volume.loc[buffer_df.dist_center ==
                                 i] = np.nansum(inter_buffer.volume)
            buffer_df.footprint.loc[buffer_df.dist_center ==
                                    i] = np.nansum(inter_buffer.buildings_area)

        for i in (np.arange(max_dist, 1, -1)):
            buffer_df.volume.loc[buffer_df.dist_center == i] = buffer_df.volume.loc[buffer_df.dist_center == i].squeeze(
            ) - buffer_df.volume.loc[buffer_df.dist_center == i-1].squeeze()
            buffer_df.area_buffer.loc[buffer_df.dist_center == i] = buffer_df.area_buffer.loc[buffer_df.dist_center == i].squeeze(
            ) - buffer_df.area_buffer.loc[buffer_df.dist_center == i-1].squeeze()
            buffer_df.footprint.loc[buffer_df.dist_center == i] = buffer_df.footprint.loc[buffer_df.dist_center == i].squeeze(
            ) - buffer_df.footprint.loc[buffer_df.dist_center == i-1].squeeze()

        buffer_df["gradient_volume"] = buffer_df.volume / buffer_df.area_buffer
        buffer_df["gradient_footprint"] = buffer_df.footprint / buffer_df.area_buffer

        model = linear_model.LinearRegression()
        results_model = model.fit(np.array(buffer_df.dist_center).reshape(-1, 1)[buffer_df["gradient_volume"].astype(float) > 0], np.log(buffer_df["gradient_volume"].astype(
            float))[buffer_df["gradient_volume"].astype(float) > 0], buffer_df["gradient_volume"].astype(float)[buffer_df["gradient_volume"].astype(float) > 0])
        volume_profile = results_model.coef_

        model = linear_model.LinearRegression()
        results_model = model.fit(np.array(buffer_df.dist_center).reshape(-1, 1)[buffer_df["gradient_footprint"].astype(float) > 0], np.log(buffer_df["gradient_footprint"].astype(
            float))[buffer_df["gradient_footprint"].astype(float) > 0], (buffer_df["gradient_footprint"].astype(float))[buffer_df["gradient_footprint"].astype(float) > 0])
        footprint_profile = results_model.coef_

        # SAVE

        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'volume_profile'] = volume_profile
        urban_form_metrics.loc[urban_form_metrics.index ==
                               city_name, 'footprint_profile'] = footprint_profile

        gdf_centroid = copy.deepcopy(gdf)
        gdf_centroid["geometry"] = gdf_centroid.centroid

        coef_radius = (UA.squeeze() / np.nansum(gdf_centroid["volume"]))

        gdf_centroid["areas_for_spag"] = gdf_centroid["volume"] * coef_radius
        gdf_centroid["radius_spag"] = np.sqrt(
            gdf_centroid["areas_for_spag"] / np.pi)

        gdf_centroid['geometry'] = gdf_centroid.buffer(
            gdf_centroid['radius_spag'], resolution=16)

        spag = gdf_centroid.dissolve()
        agglo_spag = spag.area.squeeze() / UA.squeeze()

        # SAVE

        urban_form_metrics.loc[urban_form_metrics.index ==
                           city_name, 'agglo_spag'] = agglo_spag

    urban_form_metrics.to_excel(path_folder + 'urban_form_metrics_UA.xlsx')


PL511C1.shp
1
2
3


C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.area_buffer.loc[buffer_df.dist_center == i] = buffer[0].intersection(
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:125: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3035
Right CRS: None

  inter_buffer = sjoin(gdf, gpd.GeoDataFrame(
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

4
5


C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.volume.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.footprint.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bu

6
7


...

7


C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.volume.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.footprint.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bu

8


C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.volume.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buffer_df.footprint.loc[buffer_df.dist_center ==
C:\Users\charl\AppData\Local\Temp\ipykernel_2500\3305280980.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bu

UK002C1.shp
UK003C1.shp
UK004C1.shp
UK005C1.shp
UK006C1.shp
UK007C1.shp
UK008C1.shp
UK009C1.shp
UK010C1.shp
UK011C1.shp
UK012C2.shp
UK013C1.shp
UK014C1.shp
UK016C1.shp
UK017C1.shp
UK018C1.shp
UK019C1.shp
UK020C1.shp
UK021C1.shp
UK022C1.shp
UK023C1.shp
UK024C1.shp
UK025C1.shp
UK026C1.shp
UK027C1.shp
UK028C1.shp
UK029C1.shp
UK030C1.shp
UK031C1.shp
UK032C1.shp
UK033C1.shp
UK034C1.shp
UK035C1.shp
UK036C1.shp
UK038C1.shp
UK040C1.shp
UK041C1.shp
UK043C1.shp
UK044C1.shp
UK045C1.shp
UK046C1.shp
UK047C1.shp
UK050C1.shp
UK051C1.shp
UK052C1.shp
UK053C1.shp
UK054C1.shp
UK055C1.shp
UK056C1.shp
UK057C1.shp
UK059C1.shp
UK060C1.shp
UK061C1.shp
UK062C1.shp
UK101C1.shp
UK102C1.shp
UK103C1.shp
UK104C1.shp
UK105C1.shp
UK106C1.shp
UK107C1.shp
UK108C1.shp
UK109C1.shp
UK110C1.shp
UK111C1.shp
UK112C1.shp
UK113C1.shp
UK114C1.shp
UK115C1.shp
UK116C1.shp
UK117C1.shp
UK118C1.shp
UK119C1.shp
UK120C1.shp
UK121C1.shp
UK122C1.shp
UK123C1.shp
UK124C1.shp
UK125C1.shp
UK126C1.shp
UK127C1.shp
UK128C1.shp
UK129C1.shp
UK13

In [49]:
buffer_df.dist_center


Series([], Name: dist_center, dtype: int64)